# __User Tasks__
The frontend will also handle a lot of validation and error handling, so pretend this is the best-case scenario. We can walk through the logical order of things on the frontend. 

In [1]:
# Setup 

import sys
sys.path.append('/Users/geneva/Downloads/critics')
import os
from dotenv import load_dotenv
import firebase_admin
from firebase_admin import credentials

import sys
sys.path.append('/Users/geneva/Downloads/critics/utils')

# Load environment variables from .env file
load_dotenv()

def initialize_firebase():
    service_account_path = os.getenv('SERVICE_ACCOUNT_PATH')
    database_url = os.getenv('DATABASE_URL')

    if not service_account_path:
        raise ValueError("SERVICE_ACCOUNT_PATH environment variable is not set.")

    if not firebase_admin._apps:  # Prevent duplicate initialization
        cred = credentials.Certificate(service_account_path)
        firebase_admin.initialize_app(cred, {
            'databaseURL': database_url
        })

# Set the correct path to your service account JSON file
os.environ['SERVICE_ACCOUNT_PATH'] = '/Users/geneva/Downloads/critics/gcreds_test.json'


initialize_firebase()



In [2]:
# New user is created when they first open the app. 


from utils.user import create_user

name = "Geneva"
user_id = "user_004"  

new_user = create_user(name, user_id)
print(f"Created new user: {new_user}")

Created new user: {'name': 'Geneva', 'boards': [], 'last_active': None}


In [5]:
# User creates a new board, and names it "This is a test board"

from utils.board import create_board


board_id = "board_001"  
board_name = "Genny's Board"

#create the board 
new_board = create_board(board_id, board_name, user_id) # user_id from the prev. block

print(f"Created new board: {new_board}")


ModuleNotFoundError: No module named 'utils'

In [12]:
from utils.user import join_board
# Assuming the necessary imports and setup have been done
user_id = "user_001"
board_code = "board_001"

# Call the join_board function
updated_boards = join_board(user_id, board_code)

# Print the updated list of boards for confirmation
print(f"User {user_id} is now part of the following boards: {updated_boards}")

User user_001 is now part of the following boards: ['board_001']
